In [2]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier
from tqdm import tqdm
import re
from sklearn.preprocessing import LabelEncoder
import pickle
import os
import torch
import platform
from sentence_transformers import SentenceTransformer

In [3]:
!nvidia-smi

Mon Dec  1 05:43:30 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.95.05              Driver Version: 580.95.05      CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 5090        On  |   00000000:42:00.0 Off |                  N/A |
| 37%   45C    P8             14W /  575W |       0MiB /  32607MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# Data Analysis

In [4]:
rule_col = "rule"
body_col = "body"  
label_col = "rule_violation"

In [5]:
train_df = pd.read_csv("data/train.csv")
test_df = pd.read_csv("data/test.csv")  
solution_df = pd.read_csv("data/solution.csv")

In [7]:
train_df.info()
train_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2029 entries, 0 to 2028
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   row_id              2029 non-null   int64 
 1   body                2029 non-null   object
 2   rule                2029 non-null   object
 3   subreddit           2029 non-null   object
 4   positive_example_1  2029 non-null   object
 5   positive_example_2  2029 non-null   object
 6   negative_example_1  2029 non-null   object
 7   negative_example_2  2029 non-null   object
 8   rule_violation      2029 non-null   int64 
dtypes: int64(2), object(7)
memory usage: 142.8+ KB


,row_id,body,rule,subreddit,positive_example_1,positive_example_2,negative_example_1,negative_example_2,rule_violation
0,0,Banks don't want you to know this! Click here ...,"No Advertising: Spam, referral links, unsolici...",Futurology,If you could tell your younger self something ...,hunt for lady for jack off in neighbourhood ht...,Watch Golden Globe Awards 2017 Live Online in ...,"DOUBLE CEE x BANDS EPPS - ""BIRDS""\n\nDOWNLOAD/...",0
1,1,SD Stream [ ENG Link 1] (http://www.sportsstre...,"No Advertising: Spam, referral links, unsolici...",soccerstreams,[I wanna kiss you all over! Stunning!](http://...,LOLGA.COM is One of the First Professional Onl...,#Rapper \n🚨Straight Outta Cross Keys SC 🚨YouTu...,[15 Amazing Hidden Features Of Google Search Y...,0
2,2,Lol. Try appealing the ban and say you won't d...,No legal advice: Do not offer or request legal...,pcmasterrace,Don't break up with him or call the cops. If ...,It'll be dismissed: https://en.wikipedia.org/w...,Where is there a site that still works where y...,Because this statement of his is true. It isn'...,1
3,3,she will come your home open her legs with an...,"No Advertising: Spam, referral links, unsolici...",sex,Selling Tyrande codes for 3€ to paypal. PM. \n...,tight pussy watch for your cock get her at thi...,NSFW(obviously) http://spankbang.com/iy3u/vide...,Good News ::Download WhatsApp 2.16.230 APK for...,1
4,4,code free tyrande --->>> [Imgur](http://i.imgu...,"No Advertising: Spam, referral links, unsolici...",hearthstone,wow!! amazing reminds me of the old days.Well...,seek for lady for sex in around http://p77.pl/...,must be watch movie https://sites.google.com/s...,We're streaming Pokemon Veitnamese Crystal RIG...,1


In [8]:
test_df.info()
test_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54059 entries, 0 to 54058
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   row_id              54059 non-null  int64 
 1   body                54059 non-null  object
 2   rule                54059 non-null  object
 3   subreddit           54059 non-null  object
 4   positive_example_1  54059 non-null  object
 5   positive_example_2  54059 non-null  object
 6   negative_example_1  54059 non-null  object
 7   negative_example_2  54059 non-null  object
dtypes: int64(1), object(7)
memory usage: 3.3+ MB


,row_id,body,rule,subreddit,positive_example_1,positive_example_2,negative_example_1,negative_example_2
0,2029,Hi Ken l! /u/Whimsical_wispers told me he want...,No promotion of illegal activity: Do not encou...,IAmA,There are several web sites on internet that a...,Someone needs to rape that judge and see how w...,call whores watch for you herein http://onj.me...,52 http://MySexFind.com - find girl for sex no...
1,2030,Doubt anyone will read this but anti depressan...,No medical advice: Do not offer or request spe...,science,Maybe he just has alzheimer's. It would explai...,Hmm well I WAS going to ask women of Reddit ho...,You could always just carry your kid and get t...,Maybe you should get started on the solution a...
2,2031,**[One of her best show](http://shortora.com/i...,"No Advertising: Spam, referral links, unsolici...",NSFW_GIF,Son Dakika Ek Kazanç Videolarımızı İzleyin :\n...,Try My New Game..! :)\nhttps://play.google.com...,**HD** Stream[English Tiwtch](http://www.zifoo...,/r/krat0m <-subscribe for free kilo of strong ...
3,2032,If he'd cheat on you he'll cheat on her. She's...,No spoilers: Do not reveal important details t...,relationships,And now Jessica is gone. They really need to ...,"Probably a meme at this point, particularly th...","hahah. Please, Barry changed his name to hide ...",I found the full video.\nThere were a few peop...
4,2033,I think I remember reading somewhere that tech...,No legal advice: Do not offer or request legal...,AskReddit,"I mean, even if they didn't have anything sign...","Fire her. You can't be sued for it, only the c...",Personally. I'd be wrong to tell you suicide i...,"I call it salary, perhaps a better term is sch..."


In [14]:
print("\nTraining Set")
print(f"Total samples: {len(train_df)}")
print(f"Class distribution:\n{train_df[label_col].value_counts()}")
print(f"Class ratio: {train_df[label_col].value_counts(normalize=True)}")
print(f"\nUnique rules in train: {train_df[rule_col].nunique()}")
print(f"Rules: {train_df[rule_col].unique()}")
print(f"\nUnique subreddits in train: {train_df['subreddit'].nunique()}")

print("\nTest Set")
print(f"Total samples: {len(test_df)}")
print(f"\nUnique rules in test: {test_df[rule_col].nunique()}")
print(f"Rules: {test_df[rule_col].unique()}")
print(f"\nUnique subreddits in test: {test_df['subreddit'].nunique()}")

print("\nAnalysis")
train_rules = set(train_df[rule_col].unique())
test_rules = set(test_df[rule_col].unique())
print(f"Rules in test but NOT in train: {test_rules - train_rules}")
print(f"Number of test samples with unseen rules: {test_df[~test_df[rule_col].isin(train_rules)].shape[0]}")
print(f"Percentage: {100 * test_df[~test_df[rule_col].isin(train_rules)].shape[0] / len(test_df):.2f}%")

train_subreddits = set(train_df['subreddit'].unique())
test_subreddits = set(test_df['subreddit'].unique())


Training Set
Total samples: 2029
Class distribution:
rule_violation
1    1031
0     998
Name: count, dtype: int64
Class ratio: rule_violation
1    0.508132
0    0.491868
Name: proportion, dtype: float64

Unique rules in train: 2
Rules: ['No Advertising: Spam, referral links, unsolicited advertising, and promotional content are not allowed.'
 'No legal advice: Do not offer or request legal advice.']

Unique subreddits in train: 100

Test Set
Total samples: 54059

Unique rules in test: 6
Rules: ['No promotion of illegal activity: Do not encourage or promote illegal activities, such as drug-related activity, violence, exploitation, theft, or other criminal behavior.'
 'No medical advice: Do not offer or request specific medical advice, diagnoses, or treatment recommendations.'
 'No Advertising: Spam, referral links, unsolicited advertising, and promotional content are not allowed.'
 "No spoilers: Do not reveal important details that would limit people's ability to enjoy a show or movie."

During our data analysis we noticed a few pain points in solving this problem. First, we have a small training dataset of 2,029 samples, and we are evaluating this on a test set of 54,059 samples. Secondly, we have an additional 4 rules in our test dataset that are unseen in our training dataset, these unseen rules make up 66.76% of our test samples

# Training Helper Functions

In [15]:
if torch.cuda.is_available():
    device = "cuda"
elif platform.system() == "Darwin" and torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

print(f"Using device: {device}")

Using device: cuda


In [16]:
model = SentenceTransformer("./e5-large-v2-triplet", device="cuda")

def embed_batch(texts):
    return model.encode(
        texts,
        normalize_embeddings=True,
        show_progress_bar=True
    )

In [17]:
def load_or_create_embeddings(df, prefix, cache_dir="embeddings_cache_triplet"):
    os.makedirs(cache_dir, exist_ok=True)
    embeddings = {}

    to_embed = {
        'body': body_col,
        'pos1': 'positive_example_1',
        'pos2': 'positive_example_2',
        'neg1': 'negative_example_1',
        'neg2': 'negative_example_2'
    }

    for key, col in to_embed.items():
        cache_file = f"{cache_dir}/{prefix}_{key}_emb.pkl"

        if os.path.exists(cache_file):
            print(f"Loading cached {prefix} {key} embeddings...")
            embeddings[f"{key}_emb"] = pickle.load(open(cache_file, 'rb'))
        else:
            print(f"Embedding {prefix} {col}...")
            embeddings[f"{key}_emb"] = embed_batch(df[col].tolist())
            pickle.dump(embeddings[f"{key}_emb"], open(cache_file, 'wb'))

    return embeddings



In [18]:

def extract_text_features(text):
    num_chars = len(text)
    num_words = len(text.split())

    has_url = 1 if re.search(r'http[s]?://|www\.', text) else 0

    if num_chars > 0:
        caps_ratio = sum(1 for c in text if c.isupper()) / num_chars
    else:
        caps_ratio = 0

    return np.array([num_chars, num_words, has_url, caps_ratio])

In [19]:
def combine_features(body, rule, pos1, pos2, neg1, neg2, text, subreddit_encoded):
    sim_pos1 = (body * pos1).sum()
    sim_pos2 = (body * pos2).sum()
    sim_neg1 = (body * neg1).sum()
    sim_neg2 = (body * neg2).sum()
    sim_rule = (body * rule).sum()

    pos_avg = (sim_pos1 + sim_pos2) / 2
    neg_avg = (sim_neg1 + sim_neg2) / 2
    pos_neg_ratio = pos_avg / (neg_avg + 1e-6)
    pos_neg_diff = pos_avg - neg_avg

    text_feats = extract_text_features(text)

    return np.concatenate([
        [sim_pos1, sim_pos2, sim_neg1, sim_neg2, sim_rule],  # 5 features
        [pos_neg_ratio, pos_neg_diff],  # 2 features
        text_feats,  # 4 features (num_chars, num_words, has_url, caps_ratio)
        [subreddit_encoded]  # 1 feature
    ])

In [25]:
print("Embedding unique rules...")
all_unique_rules = pd.concat([train_df[rule_col], test_df[rule_col]]).unique()
unique_rule_emb = embed_batch(all_unique_rules.tolist())
rule_to_emb = dict(zip(all_unique_rules, unique_rule_emb))
rule_emb = np.array([rule_to_emb[rule] for rule in train_df[rule_col]])

print("\nEmbedding train set")
train_emb = load_or_create_embeddings(train_df, prefix='train')
body_emb = train_emb['body_emb']
pos1_emb = train_emb['pos1_emb']
pos2_emb = train_emb['pos2_emb']
neg1_emb = train_emb['neg1_emb']
neg2_emb = train_emb['neg2_emb']
rule_emb_test = np.array([rule_to_emb[rule] for rule in test_df[rule_col]])

print("\nEmbedding test set")
test_emb = load_or_create_embeddings(test_df, prefix='test')
body_emb_test = test_emb['body_emb']
pos1_emb_test = test_emb['pos1_emb']
pos2_emb_test = test_emb['pos2_emb']
neg1_emb_test = test_emb['neg1_emb']
neg2_emb_test = test_emb['neg2_emb']

Embedding unique rules...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Embedding train set
Loading cached train body embeddings...
Loading cached train pos1 embeddings...
Loading cached train pos2 embeddings...
Loading cached train neg1 embeddings...
Loading cached train neg2 embeddings...

Embedding test set
Loading cached test body embeddings...
Loading cached test pos1 embeddings...
Loading cached test pos2 embeddings...
Loading cached test neg1 embeddings...
Loading cached test neg2 embeddings...


In [28]:
subreddit_encoder = LabelEncoder()
subreddit_encoder.fit(pd.concat([train_df['subreddit'], test_df['subreddit']]))

LabelEncoder()

In [30]:
train_subreddit_encoded = subreddit_encoder.transform(train_df['subreddit'])
y = train_df[label_col].values

X = np.array([
    combine_features(
        body_emb[i], rule_emb[i], 
        pos1_emb[i], pos2_emb[i], 
        neg1_emb[i], neg2_emb[i],
        train_df[body_col].iloc[i],
        train_subreddit_encoded[i]
    ) 
    for i in range(len(train_df))
])


print(f"Feature shape: {X.shape}")
X.shape, y.shape

Feature shape: (2029, 12)


((2029, 12), (2029,))

In [32]:
test_subreddit_encoded = subreddit_encoder.transform(test_df['subreddit'])

X_test = np.array([
    combine_features(
        body_emb_test[i], rule_emb_test[i],
        pos1_emb_test[i], pos2_emb_test[i],
        neg1_emb_test[i], neg2_emb_test[i],
        test_df[body_col].iloc[i],
        test_subreddit_encoded[i]
    )
    for i in range(len(test_df))
])

print(f"Test features shape: {X_test.shape}")

Test features shape: (54059, 12)


# Main Training Loop

In [ ]:
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000, n_jobs=-1, class_weight="balanced"),
    'Random Forest': RandomForestClassifier(n_estimators=200, max_depth=10, n_jobs=-1, class_weight="balanced", random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=200, max_depth=5, learning_rate=0.1, random_state=42),
    'XGBoost': XGBClassifier(n_estimators=200, max_depth=4, learning_rate=0.05, subsample=0.8, colsample_bytree=0.8, reg_alpha=1.0, reg_lambda=1.0, random_state=42, n_jobs=-1, eval_metric='logloss'),
    'SVM (RBF)': SVC(kernel='rbf', probability=True, class_weight='balanced', random_state=42)
}

results = {}

for name, model in models.items():
    print(f"Training {name}...")

    model.fit(X, y)
    
    test_preds = model.predict(X_test)
    test_probs = model.predict_proba(X_test)[:,1]
    
    test_results = test_df[["row_id"]].copy()
    test_results["prediction"] = test_probs
    test_results = test_results.merge(solution[["row_id", "rule_violation"]], on="row_id", how="left")
    y_test = test_results["rule_violation"].values
    
    print(f"Metrics on test dataset")
    print(classification_report(y_test, test_preds))
    test_auc = roc_auc_score(y_test, test_probs)
    test_acc = (test_preds == y_test).mean()
    print(f"AUC: {test_auc:.4f}")
    print(f"Accuracy: {test_acc:.4f}")

    
    results[name] = {
        'test_auc': test_auc,
        'test_acc': test_acc,
        'model': model,
        'test_probs': test_probs
    }
    print("\n")

print("Summary")
for name, res in results.items():
    print(f"{name:20s} | Test AUC: {res['test_auc']:.4f} | Test Acc: {res['test_acc']:.4f}")

# best_model_name = max(results.items(), key=lambda x: x[1]['test_auc'])[0]
# best_probs = results[best_model_name]['test_probs']
# test[["row_id"]].assign(prediction=best_probs).to_csv("submission.csv", index=False)
# print(f"\nSaved submission.csv using {best_model_name}")

In [116]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
import numpy as np
import random

# ==========================================
# Global Seed (Better Reproducibility)
# ==========================================

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    if torch.backends.mps.is_available():
        torch.mps.manual_seed(seed)

set_seed(42)

# ==========================================
# Improved Architecture with Residual Blocks
# ==========================================

class ResidualBlock(nn.Module):
    def __init__(self, dim, dropout=0.3):
        super().__init__()
        self.block = nn.Sequential(
            nn.Linear(dim, dim),
            nn.ReLU(),
            nn.BatchNorm1d(dim),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return x + self.block(x)  # skip connection


class RuleViolationNet(nn.Module):
    def __init__(self, input_dim=12, hidden_dims=[64, 32, 16], dropout=0.3):
        super().__init__()

        layers = []
        prev_dim = input_dim

        for hidden_dim in hidden_dims:
            layers += [
                nn.Linear(prev_dim, hidden_dim),
                nn.ReLU(),
                nn.BatchNorm1d(hidden_dim),
                nn.Dropout(dropout),

                # add a residual block for each hidden dimension
                ResidualBlock(hidden_dim, dropout)
            ]
            prev_dim = hidden_dim

        # Note: NO Sigmoid here — BCEWithLogitsLoss is better
        self.output_layer = nn.Linear(prev_dim, 1)

        self.network = nn.Sequential(*layers)

    def forward(self, x):
        x = self.network(x)
        return self.output_layer(x)  # logits (not sigmoid)


# ==========================================
# Dataset Wrapper
# ==========================================

class RuleViolationDataset(Dataset):
    def __init__(self, X, y=None):
        self.X = torch.FloatTensor(X)
        self.y = None
        if y is not None:
            self.y = torch.FloatTensor(y).unsqueeze(1)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        if self.y is not None:
            return self.X[idx], self.y[idx]
        return self.X[idx]


# ==========================================
# Neural Network Training
# ==========================================

from sklearn.model_selection import GroupShuffleSplit

def train_nn_fixed(X, y, groups, epochs=80, batch_size=32):

    gss = GroupShuffleSplit(test_size=0.2, n_splits=1, random_state=42)
    train_idx, val_idx = next(gss.split(X, y, groups))

    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]

    # scaler
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_val = scaler.transform(X_val)

    train_ds = RuleViolationDataset(X_train, y_train)
    val_ds   = RuleViolationDataset(X_val, y_val)

    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
    val_loader   = DataLoader(val_ds, batch_size=batch_size)

    # simple model
    model = RuleViolationNet(input_dim=X.shape[1], hidden_dims=[64, 32], dropout=0.4).to(device)

    # class balancing
    pos_weight = torch.tensor([(y_train.mean() / (1 - y_train.mean()))]).to(device)
    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

    optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=0.01)

    scheduler = torch.optim.lr_scheduler.OneCycleLR(
        optimizer,
        max_lr=0.003,
        steps_per_epoch=len(train_loader),
        epochs=epochs
    )

    best_auc = 0
    patience = 12
    patience_counter = 0

    for epoch in range(epochs):
        model.train()
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)

            optimizer.zero_grad()
            logits = model(xb)
            loss = criterion(logits, yb)
            loss.backward()
            optimizer.step()
            scheduler.step()

        # validation
        model.eval()
        preds, labs = [], []
        with torch.no_grad():
            for xb, yb in val_loader:
                xb = xb.to(device)
                logits = model(xb)
                preds.extend(torch.sigmoid(logits).cpu().numpy())
                labs.extend(yb.numpy())

        auc = roc_auc_score(labs, preds)

        if auc > best_auc:
            best_auc = auc
            patience_counter = 0
            best_model_state = model.state_dict()
        else:
            patience_counter += 1

        if patience_counter >= patience:
            break

    model.load_state_dict(best_model_state)
    return model, scaler, best_auc

# ==========================================
# Prediction
# ==========================================

def predict_neural_network(model, scaler, X_test, batch_size=32):
    if torch.cuda.is_available():
        device = torch.device("cuda")
    elif torch.backends.mps.is_available():
        device = torch.device("mps")
    else:
        device = torch.device("cpu")

    model.eval()

    X_test_scaled = scaler.transform(X_test)
    test_dataset = RuleViolationDataset(X_test_scaled)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)

    preds = []

    with torch.no_grad():
        for batch_X in test_loader:
            batch_X = batch_X.to(device)
            logits = model(batch_X)
            preds.extend(torch.sigmoid(logits).cpu().numpy())

    return np.array(preds).flatten()

print("Improved neural network system loaded.")
print("Training Neural Network...")

# You need grouping to avoid leakage:
groups = train["subreddit"].astype(str).values   # or "rule"

# Train model using improved fixed NN trainer
nn_model, nn_scaler, best_val_auc = train_nn_fixed(
    X, 
    y, 
    groups,
    epochs=100,
)

print(f"Best Validation AUC during training: {best_val_auc:.4f}")

# Make predictions on test set
test_probs_nn = predict_neural_network(nn_model, nn_scaler, X_test)
test_preds_nn = (test_probs_nn > 0.5).astype(int)

# Evaluate
test_results_nn = test[["row_id"]].copy()
test_results_nn["prediction"] = test_probs_nn
test_results_nn = test_results_nn.merge(
    solution[["row_id", "rule_violation"]],
    on="row_id",
    how="left",
)
y_test_nn = test_results_nn["rule_violation"].values

print("\nMetrics on test dataset")
print(classification_report(y_test_nn, test_preds_nn))

test_auc_nn = roc_auc_score(y_test_nn, test_probs_nn)
test_acc_nn = (test_preds_nn == y_test_nn).mean()

print(f"AUC: {test_auc_nn:.4f}")
print(f"Accuracy: {test_acc_nn:.4f}")

# Add to results
results["Neural Network"] = {
    "test_auc": test_auc_nn,
    "test_acc": test_acc_nn,
    "model": nn_model,
    "test_probs": test_probs_nn,
}

print("\n\nUpdated Summary (including Neural Network)")
for name, res in results.items():
    print(f"{name:20s} | Test AUC: {res['test_auc']:.4f} | Test Acc: {res['test_acc']:.4f}")



Improved neural network system loaded.
Training Neural Network...
Best Validation AUC during training: 0.9296

Metrics on test dataset
              precision    recall  f1-score   support

           0       0.73      0.76      0.75     28789
           1       0.71      0.68      0.70     25270

    accuracy                           0.72     54059
   macro avg       0.72      0.72      0.72     54059
weighted avg       0.72      0.72      0.72     54059

AUC: 0.7707
Accuracy: 0.7239


Updated Summary (including Neural Network)
Logistic Regression  | Test AUC: 0.7709 | Test Acc: 0.7201
Random Forest        | Test AUC: 0.7653 | Test Acc: 0.6506
Gradient Boosting    | Test AUC: 0.7416 | Test Acc: 0.6259
XGBoost              | Test AUC: 0.7592 | Test Acc: 0.6425
SVM (RBF)            | Test AUC: 0.7580 | Test Acc: 0.6950
Neural Network       | Test AUC: 0.7707 | Test Acc: 0.7239


In [90]:
# Create full embedding features for training set
print("Creating full embedding features for training set...")

# Concatenate all embeddings + engineered features
X_full = np.concatenate([
    body_emb,       # 768 dims
    rule_emb,       # 768 dims  
    pos1_emb,       # 768 dims
    pos2_emb,       # 768 dims
    neg1_emb,       # 768 dims
    neg2_emb,       # 768 dims
    X               # 12 engineered features (similarities, text features, subreddit)
], axis=1)

print(f"Original feature shape: {X.shape}")
print(f"Full embedding feature shape: {X_full.shape}")
print(f"Total dimensions: {X_full.shape[1]}")

# Create full embedding features for test set
print("\nCreating full embedding features for test set...")
X_test_full = np.concatenate([
    body_emb_test,
    rule_emb_test,
    pos1_emb_test,
    pos2_emb_test,
    neg1_emb_test,
    neg2_emb_test,
    X_test
], axis=1)

print(f"Test full embedding feature shape: {X_test_full.shape}")

Creating full embedding features for training set...
Original feature shape: (2029, 12)
Full embedding feature shape: (2029, 6156)
Total dimensions: 6156

Creating full embedding features for test set...
Test full embedding feature shape: (54059, 6156)


In [91]:
# Train Improved Neural Network with Full Embeddings + Simpler Architecture
print("Training Improved Neural Network...")
print("Changes:")
print("  - Using full 4620-dimensional features (embeddings + engineered features)")
print("  - Simpler architecture: [256, 64] instead of [128, 64, 32]")
print("  - Higher dropout: 0.5 instead of 0.3 (better for small datasets)")
print("  - More regularization: weight_decay increased\n")

# Split data for validation
X_train_full, X_val_full, y_train_full, y_val_full = train_test_split(
    X_full, y, test_size=0.2, random_state=42, stratify=y
)

# Train improved model with simpler architecture
nn_model_improved, nn_scaler_improved, history_improved = train_neural_network(
    X_train_full, y_train_full,
    X_val_full, y_val_full,
    epochs=100,
    batch_size=32,
    learning_rate=0.001,
    hidden_dims=[256, 64],  # Simpler: 2 layers instead of 3
    dropout=0.5              # More aggressive dropout for small dataset
)

# Make predictions on test set
test_probs_nn_improved = predict_neural_network(nn_model_improved, nn_scaler_improved, X_test_full)
test_preds_nn_improved = (test_probs_nn_improved > 0.5).astype(int)

# Evaluate
test_results_nn_improved = test[["row_id"]].copy()
test_results_nn_improved["prediction"] = test_probs_nn_improved
test_results_nn_improved = test_results_nn_improved.merge(solution[["row_id", "rule_violation"]], on="row_id", how="left")
y_test_nn_improved = test_results_nn_improved["rule_violation"].values

print("\n" + "="*60)
print("IMPROVED NEURAL NETWORK - Metrics on test dataset")
print("="*60)
print(classification_report(y_test_nn_improved, test_preds_nn_improved))
test_auc_nn_improved = roc_auc_score(y_test_nn_improved, test_probs_nn_improved)
test_acc_nn_improved = (test_preds_nn_improved == y_test_nn_improved).mean()
print(f"AUC: {test_auc_nn_improved:.4f}")
print(f"Accuracy: {test_acc_nn_improved:.4f}")

# Add to results
results['Neural Network (Improved)'] = {
    'test_auc': test_auc_nn_improved,
    'test_acc': test_acc_nn_improved,
    'model': nn_model_improved,
    'test_probs': test_probs_nn_improved
}

print("\n" + "="*60)
print("COMPARISON: All Models")
print("="*60)
for name, res in sorted(results.items(), key=lambda x: x[1]['test_auc'], reverse=True):
    print(f"{name:30s} | Test AUC: {res['test_auc']:.4f} | Test Acc: {res['test_acc']:.4f}")

# Highlight improvement
original_nn_auc = results['Neural Network']['test_auc']
improvement = test_auc_nn_improved - original_nn_auc
print(f"\n{'='*60}")
print(f"Neural Network Improvement: {improvement:+.4f} AUC")
if test_auc_nn_improved > original_nn_auc:
    print(f"✓ {abs(improvement)/original_nn_auc*100:.1f}% relative improvement!")
print(f"{'='*60}")

Training Improved Neural Network...
Changes:
  - Using full 4620-dimensional features (embeddings + engineered features)
  - Simpler architecture: [256, 64] instead of [128, 64, 32]
  - Higher dropout: 0.5 instead of 0.3 (better for small datasets)
  - More regularization: weight_decay increased

Epoch [10/100] - Train Loss: 0.1243 - Val Loss: 0.4307 - Val AUC: 0.9380
Epoch [20/100] - Train Loss: 0.0796 - Val Loss: 0.4761 - Val AUC: 0.9433
Epoch [30/100] - Train Loss: 0.0425 - Val Loss: 0.5416 - Val AUC: 0.9405
Epoch [40/100] - Train Loss: 0.0201 - Val Loss: 0.5936 - Val AUC: 0.9387
Epoch [50/100] - Train Loss: 0.0223 - Val Loss: 0.6573 - Val AUC: 0.9369
Epoch [60/100] - Train Loss: 0.0276 - Val Loss: 0.7073 - Val AUC: 0.9337
Epoch [70/100] - Train Loss: 0.0232 - Val Loss: 0.7505 - Val AUC: 0.9357
Epoch [80/100] - Train Loss: 0.0106 - Val Loss: 0.8070 - Val AUC: 0.9365
Epoch [90/100] - Train Loss: 0.0036 - Val Loss: 0.7916 - Val AUC: 0.9368
Epoch [100/100] - Train Loss: 0.0141 - Val Lo

In [95]:
# ================================================================
# FULL PIPELINE — Triplet Embeddings → Neural Network Classifier
# ================================================================

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import roc_auc_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupShuffleSplit
from sentence_transformers import SentenceTransformer

# ===============================
# 1. Device selection
# ===============================
if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

print("Using:", device)

# ===============================
# 2. Load Triplet-Tuned Embedding Model
# ===============================
print("Loading triplet model...")
model_triplet = SentenceTransformer("./e5-large-v2-triplet", device=device)

# ===============================
# 3. Embed Training + Test Data
# ===============================
print("Embedding training data...")
X_full = model_triplet.encode(
    train["body"].tolist(),
    batch_size=32,
    normalize_embeddings=True,
    show_progress_bar=True
)

print("Embedding test data...")
X_test_full = model_triplet.encode(
    test["body"].tolist(),
    batch_size=32,
    normalize_embeddings=True,
    show_progress_bar=True
)

y_full = train_df["rule_violation"].astype(int).values
groups = train_df["subreddit"].astype(str).values  # Group split to avoid leakage

print("Train embedding shape:", X_full.shape)
print("Test embedding shape:", X_test_full.shape)


# ================================================================
# 4. Dataset & Model Definitions
# ================================================================
class RuleViolationDataset(Dataset):
    def __init__(self, X, y=None):
        self.X = torch.FloatTensor(X)
        self.y = None if y is None else torch.FloatTensor(y).unsqueeze(1)

    def __len__(self): return len(self.X)

    def __getitem__(self, idx):
        if self.y is None:
            return self.X[idx]
        return self.X[idx], self.y[idx]


class RuleViolationNet(nn.Module):
    def __init__(self, input_dim=1024, dropout=0.4):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(128, 1)
        )

    def forward(self, x): return self.net(x)


# ================================================================
# 5. Improved Neural Network Trainer
# ================================================================
def train_nn_fixed(X, y, groups, epochs=80, batch_size=32):

    gss = GroupShuffleSplit(test_size=0.2, n_splits=1, random_state=42)
    train_idx, val_idx = next(gss.split(X, y, groups))

    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_val = scaler.transform(X_val)

    train_ds = RuleViolationDataset(X_train, y_train)
    val_ds = RuleViolationDataset(X_val, y_val)

    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_ds, batch_size=batch_size)

    model = RuleViolationNet(input_dim=X.shape[1]).to(device)

    pos_weight = torch.tensor([(y_train.mean() / (1 - y_train.mean()))]).to(device)
    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

    optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=0.01)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(
        optimizer, max_lr=0.003,
        steps_per_epoch=len(train_loader),
        epochs=epochs
    )

    best_auc = 0
    patience = 12
    no_improve = 0

    for epoch in range(epochs):
        model.train()
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            optimizer.zero_grad()
            logits = model(xb)
            loss = criterion(logits, yb)
            loss.backward()
            optimizer.step()
            scheduler.step()

        # validation
        model.eval()
        preds, labels = [], []
        with torch.no_grad():
            for xb, yb in val_loader:
                xb = xb.to(device)
                logits = model(xb)
                preds.extend(torch.sigmoid(logits).cpu().numpy())
                labels.extend(yb.numpy())

        auc = roc_auc_score(labels, preds)
        if auc > best_auc:
            best_auc = auc
            no_improve = 0
            best_state = model.state_dict()
        else:
            no_improve += 1

        if no_improve >= patience:
            break

    model.load_state_dict(best_state)
    return model, scaler, best_auc


# ================================================================
# 6. Train Neural Network on Full Triplet Embeddings
# ================================================================
print("Training Neural Network (Triplet Embeddings)...")
nn_model, nn_scaler, best_val_auc = train_nn_fixed(X_full, y_full, groups, epochs=80)

print(f"Best Validation AUC: {best_val_auc:.4f}")


# ================================================================
# 7. Predict on Test Set
# ================================================================
def predict_nn(model, scaler, X):
    X_scaled = scaler.transform(X)
    ds = RuleViolationDataset(X_scaled)
    loader = DataLoader(ds, batch_size=64)

    preds = []
    model.eval()
    with torch.no_grad():
        for xb in loader:
            xb = xb.to(device)
            logits = model(xb)
            preds.extend(torch.sigmoid(logits).cpu().numpy())
    return np.array(preds).flatten()


print("Predicting on test set...")
test_probs_nn = predict_nn(nn_model, nn_scaler, X_test_full)
test_preds_nn = (test_probs_nn > 0.5).astype(int)

# Evaluate on "solution"
test_results = test[["row_id"]].copy()
test_results["prediction"] = test_probs_nn
test_results = test_results.merge(solution[["row_id", "rule_violation"]], on="row_id")
y_test_true = test_results["rule_violation"].values

print("\nFinal Test Metrics:")
print(classification_report(y_test_true, test_preds_nn))

auc = roc_auc_score(y_test_true, test_probs_nn)
acc = (test_preds_nn == y_test_true).mean()

print(f"AUC: {auc:.4f}")
print(f"Accuracy: {acc:.4f}")


Using: cuda
Loading triplet model...
Embedding training data...


Batches:   0%|          | 0/64 [00:00<?, ?it/s]

Embedding test data...


Batches:   0%|          | 0/1690 [00:00<?, ?it/s]

Train embedding shape: (2029, 1024)
Test embedding shape: (54059, 1024)
Training Neural Network (Triplet Embeddings)...
Best Validation AUC: 0.9443
Predicting on test set...

Final Test Metrics:
              precision    recall  f1-score   support

           0       0.62      0.76      0.68     28789
           1       0.63      0.46      0.53     25270

    accuracy                           0.62     54059
   macro avg       0.62      0.61      0.61     54059
weighted avg       0.62      0.62      0.61     54059

AUC: 0.6329
Accuracy: 0.6214


In [17]:
# Install transformers if needed
import sys
!{sys.executable} -m pip install -q transformers torch

from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
import warnings
warnings.filterwarnings('ignore')

print("NLI libraries loaded successfully!")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


NLI libraries loaded successfully!


In [20]:
# Load Zero-Shot NLI model
print("Loading BART-Large MNLI model...")
print("(This model is specifically trained on Multi-NLI for entailment/contradiction)")
print("Note: Using BART instead of DeBERTa for better compatibility\n")

nli_model_name = "facebook/bart-large-mnli"
nli_tokenizer = AutoTokenizer.from_pretrained(nli_model_name)
nli_model = AutoModelForSequenceClassification.from_pretrained(nli_model_name)

# Move to MPS if available
if torch.backends.mps.is_available():
    nli_model = nli_model.to('mps')
    print("✓ Model loaded on MPS device")
elif torch.cuda.is_available():
    nli_model = nli_model.to('cuda')
    print("✓ Model loaded on CUDA device")
else:
    print("✓ Model loaded on CPU")

print("✓ NLI model ready!")
print(f"  Model size: ~560M parameters")
print(f"  Perfect for zero-shot classification on unseen rules!")

Loading BART-Large MNLI model...
(This model is specifically trained on Multi-NLI for entailment/contradiction)
Note: Using BART instead of DeBERTa for better compatibility

✓ Model loaded on CUDA device
✓ NLI model ready!
  Model size: ~560M parameters
  Perfect for zero-shot classification on unseen rules!


In [21]:
def get_nli_scores(premises, hypotheses, batch_size=16):
    """
    Compute NLI scores for premise-hypothesis pairs.
    
    For rule violation detection:
    - premise: "This text violates the rule: {rule}"
    - hypothesis: "{body}"
    
    Returns:
    - entailment score (higher = more likely a violation)
    - contradiction score (higher = less likely a violation)
    - neutral score
    """
    device = nli_model.device
    all_entailment_scores = []
    all_contradiction_scores = []
    all_neutral_scores = []
    
    print(f"Processing {len(premises)} samples in batches of {batch_size}...")
    
    for i in tqdm(range(0, len(premises), batch_size)):
        batch_premises = premises[i:i+batch_size]
        batch_hypotheses = hypotheses[i:i+batch_size]
        
        # Tokenize
        inputs = nli_tokenizer(
            batch_premises,
            batch_hypotheses,
            truncation=True,
            padding=True,
            max_length=512,
            return_tensors="pt"
        ).to(device)
        
        # Get predictions
        with torch.no_grad():
            outputs = nli_model(**inputs)
            logits = outputs.logits
            probs = torch.softmax(logits, dim=-1)
        
        # Extract scores (DeBERTa outputs: [contradiction, neutral, entailment])
        contradiction_scores = probs[:, 0].cpu().numpy()
        neutral_scores = probs[:, 1].cpu().numpy()
        entailment_scores = probs[:, 2].cpu().numpy()
        
        all_contradiction_scores.extend(contradiction_scores)
        all_neutral_scores.extend(neutral_scores)
        all_entailment_scores.extend(entailment_scores)
    
    return np.array(all_entailment_scores), np.array(all_contradiction_scores), np.array(all_neutral_scores)

print("✓ NLI scoring function defined!")

✓ NLI scoring function defined!


In [22]:
print("="*80)
print("COMPUTING NLI FEATURES FOR TRAINING SET")
print("="*80)
print("\nStrategy: Frame the problem as entailment")
print("- If body ENTAILS 'this violates {rule}' → violation likely")
print("- If body CONTRADICTS 'this violates {rule}' → violation unlikely\n")

# Create NLI prompts for training set
train_premises_v1 = [f"This text violates the rule: {rule}" for rule in train[rule_col]]
train_hypotheses = train[body_col].tolist()

print("Computing direct violation scores (body vs rule)...")
train_ent_v1, train_contra_v1, train_neu_v1 = get_nli_scores(train_premises_v1, train_hypotheses, batch_size=16)

# Also check entailment with positive/negative examples
print("\nComputing similarity to positive examples...")
train_premises_pos = [f"This is an example that violates the rule: {train['positive_example_1'].iloc[i]}" 
                      for i in range(len(train))]
train_ent_pos, train_contra_pos, _ = get_nli_scores(train_premises_pos, train_hypotheses, batch_size=16)

print("\nComputing similarity to negative examples...")
train_premises_neg = [f"This is an example that does NOT violate the rule: {train['negative_example_1'].iloc[i]}" 
                      for i in range(len(train))]
train_ent_neg, train_contra_neg, _ = get_nli_scores(train_premises_neg, train_hypotheses, batch_size=16)

# Combine NLI features
X_nli_train = np.column_stack([
    train_ent_v1,          # Entailment with violation
    train_contra_v1,       # Contradiction with violation  
    train_neu_v1,          # Neutral
    train_ent_pos,         # Similarity to positive examples
    train_contra_pos,      # Dissimilarity to positive examples
    train_ent_neg,         # Similarity to negative examples
    train_contra_neg,      # Dissimilarity to negative examples
])

print(f"\n✓ Training NLI features shape: {X_nli_train.shape}")
print(f"  Features: entailment, contradiction, neutral, pos_sim, pos_dissim, neg_sim, neg_dissim")

COMPUTING NLI FEATURES FOR TRAINING SET

Strategy: Frame the problem as entailment
- If body ENTAILS 'this violates {rule}' → violation likely
- If body CONTRADICTS 'this violates {rule}' → violation unlikely

Computing direct violation scores (body vs rule)...
Processing 2029 samples in batches of 16...


100%|██████████| 127/127 [00:05<00:00, 24.40it/s]



Computing similarity to positive examples...
Processing 2029 samples in batches of 16...


100%|██████████| 127/127 [00:07<00:00, 16.60it/s]



Computing similarity to negative examples...
Processing 2029 samples in batches of 16...


100%|██████████| 127/127 [00:07<00:00, 16.98it/s]


✓ Training NLI features shape: (2029, 7)
  Features: entailment, contradiction, neutral, pos_sim, pos_dissim, neg_sim, neg_dissim


In [23]:
print("="*80)
print("COMPUTING NLI FEATURES FOR TEST SET")
print("="*80)
print("This includes 4 UNSEEN RULES (66.76% of test data)!\n")

# Create NLI prompts for test set
test_premises_v1x` = [f"This text violates the rule: {rule}" for rule in test[rule_col]]
test_hypotheses = test[body_col].tolist()

print("Computing direct violation scores (body vs rule)...")
test_ent_v1, test_contra_v1, test_neu_v1 = get_nli_scores(test_premises_v1, test_hypotheses, batch_size=16)

print("\nComputing similarity to positive examples...")
test_premises_pos = [f"This is an example that violates the rule: {test['positive_example_1'].iloc[i]}" 
                     for i in range(len(test))]
test_ent_pos, test_contra_pos, _ = get_nli_scores(test_premises_pos, test_hypotheses, batch_size=16)

print("\nComputing similarity to negative examples...")
test_premises_neg = [f"This is an example that does NOT violate the rule: {test['negative_example_1'].iloc[i]}" 
                     for i in range(len(test))]
test_ent_neg, test_contra_neg, _ = get_nli_scores(test_premises_neg, test_hypotheses, batch_size=16)

# Combine NLI features
X_nli_test = np.column_stack([
    test_ent_v1,
    test_contra_v1,
    test_neu_v1,
    test_ent_pos,
    test_contra_pos,
    test_ent_neg,
    test_contra_neg,
])

print(f"\n✓ Test NLI features shape: {X_nli_test.shape}")
print(f"  Features: entailment, contradiction, neutral, pos_sim, pos_dissim, neg_sim, neg_dissim")

SyntaxError: invalid syntax (539044655.py, line 7)

In [24]:
print("="*80)
print("TRAINING MODELS WITH NLI FEATURES")
print("="*80)

# Test 1: NLI features only
print("\n[1] Training Logistic Regression with NLI features ONLY...")
lr_nli = LogisticRegression(max_iter=1000, class_weight="balanced", random_state=42)
lr_nli.fit(X_nli_train, y)

test_probs_nli = lr_nli.predict_proba(X_nli_test)[:, 1]
test_preds_nli = (test_probs_nli > 0.5).astype(int)

# Evaluate
test_results_nli = test[["row_id"]].copy()
test_results_nli["prediction"] = test_probs_nli
test_results_nli = test_results_nli.merge(solution[["row_id", "rule_violation"]], on="row_id", how="left")
y_test_nli = test_results_nli["rule_violation"].values

print("\nMetrics on test dataset (NLI features only):")
print(classification_report(y_test_nli, test_preds_nli))
test_auc_nli = roc_auc_score(y_test_nli, test_probs_nli)
test_acc_nli = (test_preds_nli == y_test_nli).mean()
print(f"AUC: {test_auc_nli:.4f}")
print(f"Accuracy: {test_acc_nli:.4f}")

results['Logistic Regression (NLI only)'] = {
    'test_auc': test_auc_nli,
    'test_acc': test_acc_nli,
    'model': lr_nli,
    'test_probs': test_probs_nli
}

TRAINING MODELS WITH NLI FEATURES

[1] Training Logistic Regression with NLI features ONLY...


NameError: name 'X_nli_test' is not defined

In [ ]:
# Test 2: Combine NLI features with original similarity features
print("\n[2] Training with COMBINED features (NLI + Original)...")

X_combined = np.hstack([X_nli_train, X])  # 7 NLI features + 12 original = 19 features
X_test_combined = np.hstack([X_nli_test, X_test])

print(f"Combined feature shape: {X_combined.shape}")

lr_combined = LogisticRegression(max_iter=1000, class_weight="balanced", random_state=42)
lr_combined.fit(X_combined, y)

test_probs_combined = lr_combined.predict_proba(X_test_combined)[:, 1]
test_preds_combined = (test_probs_combined > 0.5).astype(int)

# Evaluate
test_results_combined = test[["row_id"]].copy()
test_results_combined["prediction"] = test_probs_combined
test_results_combined = test_results_combined.merge(solution[["row_id", "rule_violation"]], on="row_id", how="left")
y_test_combined = test_results_combined["rule_violation"].values

print("\nMetrics on test dataset (NLI + Original features):")
print(classification_report(y_test_combined, test_preds_combined))
test_auc_combined = roc_auc_score(y_test_combined, test_probs_combined)
test_acc_combined = (test_preds_combined == y_test_combined).mean()
print(f"AUC: {test_auc_combined:.4f}")
print(f"Accuracy: {test_acc_combined:.4f}")

results['Logistic Regression (NLI + Original)'] = {
    'test_auc': test_auc_combined,
    'test_acc': test_acc_combined,
    'model': lr_combined,
    'test_probs': test_probs_combined
}

In [ ]:
print("\n" + "="*80)
print("FINAL COMPARISON: All Models Including Zero-Shot NLI")
print("="*80)

# Sort by test AUC
sorted_results = sorted(results.items(), key=lambda x: x[1]['test_auc'], reverse=True)

print(f"\n{'Model':<40} {'Test AUC':<12} {'Test Acc':<12} {'Improvement':<12}")
print("-" * 80)

best_previous_auc = max([res['test_auc'] for name, res in results.items() 
                         if 'NLI' not in name])

for name, res in sorted_results:
    auc = res['test_auc']
    acc = res['test_acc']
    
    if 'NLI' in name:
        improvement = auc - best_previous_auc
        improvement_str = f"+{improvement:.4f}" if improvement > 0 else f"{improvement:.4f}"
        marker = "🎯 NEW!" if improvement > 0 else ""
        print(f"{name:<40} {auc:<12.4f} {acc:<12.4f} {improvement_str:<12} {marker}")
    else:
        print(f"{name:<40} {auc:<12.4f} {acc:<12.4f}")

print("="*80)

# Highlight the best model
best_model_name = sorted_results[0][0]
best_auc = sorted_results[0][1]['test_auc']
print(f"\n🏆 BEST MODEL: {best_model_name}")
print(f"   Test AUC: {best_auc:.4f}")

if 'NLI' in best_model_name:
    improvement = best_auc - best_previous_auc
    print(f"   Improvement over previous best: +{improvement:.4f} ({improvement/best_previous_auc*100:.2f}%)")
    print(f"\n✨ Zero-Shot NLI successfully handles UNSEEN RULES!")

In [ ]:
# Save the best NLI model's predictions
best_nli_probs = results[best_model_name]['test_probs']
submission_nli = test[["row_id"]].copy()
submission_nli["prediction"] = best_nli_probs
submission_nli.to_csv("submission_nli.csv", index=False)

print(f"\n✓ Saved submission_nli.csv using {best_model_name}")
print(f"  This model should perform MUCH better on unseen rules!")